In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, TimeDistributed, LSTM
from tensorflow.keras.optimizers import Adam


In [2]:
# -------------------------------
# Paths to dataset
# -------------------------------
train_dir = r"C:\Users\kotaa\Downloads\emotion_recognition_1\emotion_recognition\data\train"
test_dir  = r"C:\Users\kotaa\Downloads\emotion_recognition_1\emotion_recognition\data\test"


In [3]:
# -------------------------------
# Image Data Generators
# -------------------------------
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

img_height, img_width = 48, 48   # grayscale FER dataset usually 48x48
sequence_length = 10             # number of frames per sequence

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode="categorical",
    color_mode="grayscale"
)

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode="categorical",
    color_mode="grayscale"
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [4]:
# -------------------------------
# Build CNN + LSTM Model
# -------------------------------
cnn = Sequential([
    Conv2D(32, (3,3), activation="relu", input_shape=(img_height, img_width, 1)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation="relu"),
    MaxPooling2D((2,2)),
    Conv2D(128, (3,3), activation="relu"),
    MaxPooling2D((2,2)),
    Flatten()
])


In [5]:
# Wrap CNN in TimeDistributed for sequences
model = Sequential([
    TimeDistributed(cnn, input_shape=(sequence_length, img_height, img_width, 1)),
    LSTM(128, return_sequences=False),
    Dropout(0.5),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation="softmax")
])

In [6]:
# -------------------------------
# Compile Model
# -------------------------------
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation="relu", input_shape=(48, 48, 1)),  # grayscale
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation="relu"),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation="relu"),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation="softmax")
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


In [12]:
# -------------------------------
# Train Model
# -------------------------------
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=15
)

Epoch 1/15
898/898 [==============================] - 39s 43ms/step - loss: 1.6668 - accuracy: 0.3365 - val_loss: 1.5187 - val_accuracy: 0.4217
Epoch 2/15
898/898 [==============================] - 36s 40ms/step - loss: 1.4940 - accuracy: 0.4220 - val_loss: 1.3769 - val_accuracy: 0.4767
Epoch 3/15
898/898 [==============================] - 34s 38ms/step - loss: 1.3900 - accuracy: 0.4731 - val_loss: 1.3022 - val_accuracy: 0.5060
Epoch 4/15
898/898 [==============================] - 35s 39ms/step - loss: 1.3149 - accuracy: 0.5002 - val_loss: 1.2676 - val_accuracy: 0.5210
Epoch 5/15
898/898 [==============================] - 34s 38ms/step - loss: 1.2631 - accuracy: 0.5235 - val_loss: 1.2439 - val_accuracy: 0.5234
Epoch 6/15
898/898 [==============================] - 36s 40ms/step - loss: 1.2208 - accuracy: 0.5403 - val_loss: 1.2000 - val_accuracy: 0.5482
Epoch 7/15
898/898 [==============================] - 36s 40ms/step - loss: 1.1809 - accuracy: 0.5522 - val_loss: 1.1801 - val_accuracy:

In [13]:

# -------------------------------
# Evaluate Model
# -------------------------------
loss, acc = model.evaluate(validation_generator)
print(f"✅ Test Accuracy: {acc*100:.2f}%")

225/225 [==============================] - 5s 24ms/step - loss: 1.2025 - accuracy: 0.5645
✅ Test Accuracy: 56.45%


In [14]:
# -------------------------------
# Save Model
# -------------------------------
model.save("cnn_lstm_emotion.h5")
print("🎉 Hybrid CNN + LSTM Model training completed and saved!")

🎉 Hybrid CNN + LSTM Model training completed and saved!


In [15]:
model.save_weights("hybrid_model.h5")
